In [1]:
import requests
import random
import io
import os
from datetime import datetime
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
from dotenv import load_dotenv
import pdfplumber
from openai import OpenAI

load_dotenv()

True

In [2]:
base_header = {
    "authority": "finance.naver.com",
    "method": "GET",
    "path": "/research/company_list.naver?&page=1",
    "scheme": "https",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "cache-control": "no-cache",
    "cookie": "",
    "pragma": "no-cache",
    "priority": "u=0, i",
    "referer": "https://finance.naver.com/research/company_list.naver?&page=2",
    "sec-ch-ua": '"Not)A;Brand";v="8", "Chromium";v="138", "Google Chrome";v="138"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "Windows",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "same-origin",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": os.getenv("USER_AGENT")
}

In [3]:
report_list = []
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
today = datetime.now().strftime("%Y.%m.%d")

def summarize_report(report_text: str) -> str:
    try:
        resp = client.responses.create(
            model="gpt-4.1",
            input=[
                {"role": "developer",
                 "content": """당신은 한국 증권사 리포트를 요약하는 전문가입니다.
                    ### 작업 지침 ###
                    1. 아래 리포트(첫 페이지)를 읽고 핵심 내용을 **3~6문장**으로 요약하세요.
                    2. **수치(매출액·성장률 등)는 최소화**하고, 왜 그런 전망을 내놓았는지 **논리·근거** 위주로 작성하세요.
                    3. **목표주가(TP)와 투자의견**이 있으면 반드시 포함하세요.
                    4. 보고서 작성자의 면책 조항과 관련된 내용은 제외하세요.
                    5. GPT의 의견을 추가하지 말고, 철저히 글 내용 기반으로 답해주세요."""},
                {"role": "user", "content": report_text}
            ],
            temperature=0.1,
        )
        return resp.output_text.strip()
    except Exception as e:
        return f"요약 실패: {e}"

with requests.Session() as session:
    for page in range(1, 2):
        base_url = f"https://finance.naver.com/research/company_list.naver?&page={page}"
        req = session.get(base_url, headers=base_header)
        soup = BeautifulSoup(req.text, "html.parser")    
        table = soup.find("table", class_="type_1")
        if not table:
            continue
        rows = table.find_all("tr")
        for row in tqdm(rows):
            cols = row.find_all("td")
            if len(cols) < 5:
                continue
            securities = cols[2].get_text(strip=True)  # 증권사

            # 신한투자증권이면 스킵
            if securities == "신한투자증권":
                continue

            report_title = cols[1].get_text(strip=True)  # 리포트 이름
            company_name = cols[0].get_text(strip=True)  # 회사 이름

            # pdf 주소는 .pdf로 끝나는 a 태그에서 추출
            pdf_url = ""
            pdf_a_tag = row.find("a", href=lambda x: x and x.endswith(".pdf"))
            if pdf_a_tag and pdf_a_tag.get("href"):
                pdf_url = pdf_a_tag.get("href")
                # pdf_url이 절대경로가 아니면, 앞에 https://stock.pstatic.net 붙이기
                if not pdf_url.startswith("http"):
                    pdf_url = "https://stock.pstatic.net" + pdf_url

            date = cols[4].get_text(strip=True) if len(cols) > 4 else ""  # 올라온 날짜

            # 오늘 날짜가 아니면 스킵
            if date != today:
                continue

            # pdf_url이 있으면 pdf에서 텍스트 추출
            report_text = ""
            if pdf_url:
                try:
                    pdf_req = session.get(pdf_url)
                    # PDF 요청 후에도 랜덤하게 1~2초 sleep
                    time.sleep(random.uniform(1, 2))
                    pdf_file = io.BytesIO(pdf_req.content)
                    with pdfplumber.open(pdf_file) as pdf:
                        first_page = pdf.pages[0]
                        report_text = first_page.extract_text()
                except Exception as e:
                    report_text = f"PDF 추출 실패: {e}"

            summary = ""
            if report_text and not report_text.startswith("PDF 추출 실패"):
                summary = summarize_report(report_text)
            else:
                summary = "텍스트 없음 또는 PDF 추출 실패"

            report_dict = {
                "증권사": securities,
                "리포트명": report_title,
                "회사명": company_name,
                "pdf주소": pdf_url,
                "날짜": date,
                "텍스트": report_text,
                "요약": summary
            }
            report_list.append(report_dict)

100%|██████████| 49/49 [00:00<00:00, 49108.94it/s]


In [4]:
print("AI가 요약한 결과입니다. 정보의 정확성은 보장하지 않습니다.")
print()
print("================================================")
for report_dict in report_list:
    print("## ",report_dict["리포트명"], "|", report_dict["회사명"], "|", report_dict["증권사"], "|", report_dict["날짜"], " ##")
    print()
    # 요약을 문장 단위로 쪼개서 '. ' 뒤에 엔터를 넣어 출력
    summary = report_dict["요약"]
    # '. '로 끝나는 문장 뒤에만 줄바꿈 추가 (예: 7.9만원 등은 줄바꿈하지 않음)
    import re
    sentences = re.split(r'(?<=\.\s)', summary)
    for sentence in sentences:
        if sentence.strip():  # 빈 문장은 출력하지 않음
            print("◼︎ ",sentence.strip())
    print("================================================")
    print()

AI가 요약한 결과입니다. 정보의 정확성은 보장하지 않습니다.



In [ ]:
()